In [3]:
### testing stuff with dual numbers here


struct Dual <: Number
    f::Real
    g::Real
end



In [172]:
## overloading operators 
using Distributions

import Base: +,/,*,-,^, convert, promote_rule
+(x::Dual, y::Dual) = Dual(x.f + y.f, x.g + y.g)
+(x::Dual, y::Real) = Dual(x.f + y, x.g)
+(y::Real, x::Dual) = Dual(x.f + y, x.g)

-(x::Dual, y::Dual) = Dual(x.f - y.f, x.g - y.g)
-(x::Dual) = Dual(-x.f, -x.g)
-(x::Dual, y::Real) = Dual(x.f -y, x.g)
-(y::Real, x::Dual) = Dual(y-x.f, -x.g)

*(x::Dual, y::Dual) = Dual(x.f*y.f, x.f*y.g + y.f*x.g)
*(x::Dual, y::Real) = Dual(x.f*y, x.g*y)
*(y::Real, x::Dual) = Dual(x.f*y, x.g*y)

/(x::Dual, y::Dual) = Dual(x.f/y.f, (y.f*x.g - x.f*y.g)/y.f^2)
/(y::Real, x::Dual) = Dual(y/x.f, (-y*x.g) / x.f^2)
/(x::Dual, y::Real) = Dual(x.f/y, x.g/y)

^(x::Dual, k::Real) = Dual(x.f^k, (x.g * k) * x.f ^ (k-1))



Base.exp(x::Dual) = Dual(exp(x.f), x.g * exp(x.f))
Base.sqrt(x::Dual) = Dual(sqrt(x.f), x.g / (2 * sqrt(x.f)))
Base.log(x::Dual) = Dual(log(x.f), x.g/x.f)

Distributions.cdf(d, x::Dual) = Dual(cdf(d, x.f), pdf(d, x.f) * x.g)

## helper functions to convert from duals to reals
convert(::Type{Dual}, x::Real) = Dual(x, one(x))
#convert(::Type{D}, x::AbstractArray) = AbstractArray{D((x, ones(length(x)) ))}#Dual((x, zero(x)))
promote_rule(::Type{Dual}, ::Type{<:Number}) = Dual


promote_rule (generic function with 154 methods)

In [200]:

function derivative(f, x)
    return f(Dual(x,1)).g
end

derivative (generic function with 1 method)

In [165]:
using BenchmarkTools


tt (generic function with 1 method)

In [166]:
S = 178
K = 97
T = 0.47
r = 0.0112
q = 0

0

In [199]:
d = Normal()
d1(S,K,T,r,v) = (log(S/K) + (r + v*v/2)*T)/(v*sqrt(T))
d2(S,K,T,r,v) = (log(S/K) + (r + v*v/2)*T)/(v*sqrt(T)) - v*sqrt(T)
call_price(S,K,T,r,v,q) = S*exp(-q*T)*cdf(d, d1(S,K,T,r,v)) - K*exp(-r*T)*cdf(d, d2(S,K,T,r,v))
put_price(S,K,T,r,v,q) = K*exp(-r*T)*cdf(d, -d2(S,K,T,r,v)) - S*exp(-q*T)*cdf(d, -d1(S,K,T,r,v))

price_v = v -> put_price(S,K,T,r,v,q)

@btime ForwardDiff.derivative(price_v, 1)

  415.070 ns (11 allocations: 304 bytes)


22.680964292858352

In [198]:
@btime deriv(price_v, 1)

  3.587 μs (136 allocations: 2.56 KiB)


22.680964292858352

In [196]:
@btime [deriv(price_v, i) for i in 1:100]

  365.000 μs (13915 allocations: 262.06 KiB)


100-element Array{Float64,1}:
 22.680964292858352      
 25.6474224710635        
 20.208319776698787      
 13.656762890886721      
  8.121053290198851      
  4.276287286570536      
  1.9982560759614323     
  0.8293879925851797     
  0.3058963004203713     
  0.10027654544442326    
  0.029220604998773783   
  0.0075696789252974     
  0.0017433532703132106  
  ⋮                      
  2.829128666036503e-201 
  7.664853123859659e-206 
  1.846398155882849e-210 
  3.9547338524225884e-215
  7.531466506477353e-220 
  1.2752995931823724e-224
  1.9200621137236045e-229
  2.5703290285507674e-234
  3.0593738479541314e-239
  3.237775746047776e-244 
  3.0467117543827056e-249
  2.5490969790679187e-254